## Task -1 Exploratory Data Analysis

In [ ]:
"""
Installing Spark with its dependencies
Installing Spark
Install Dependencies:

Java 8
Apache Spark with hadoop and
Findspark (used to locate the spark in the system)
"""

!sudo ./install_spark.py

In [1]:
#Set Environment Variables:

import os

current_directory = os.getcwd()


os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = os.path.join(current_directory,"spark-3.1.1-bin-hadoop3.2")

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

#Create a SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("MovieLens") \
    .config("spark.jars", "spark-3.1.1-bin-hadoop3.2/jars/sqlite-jdbc-3.34.0.jar") \
    .getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

24/01/23 12:17:50 WARN Utils: Your hostname, codespaces-7047d6 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/01/23 12:17:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/01/23 12:17:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
#will be used sqlite3 to be able to reach .db file

import sqlite3

con = sqlite3.connect('Datasets/movielens-small.db')
cur = con.cursor()

In [4]:
# Get the field information of each table in the database

# Define a function to print table information in a readable way
def print_table_info(table_name, table_info):
    print(f"\nTable: {table_name}")
    for row in table_info:
        print(row[0], row[1], row[2],sep=": ")

# Get and print information for the 'movies' table
cur.execute("PRAGMA table_info(movies);")
movies_info = cur.fetchall()
print_table_info("movies", movies_info)

# Get and print information for the 'ratings' table
cur.execute("PRAGMA table_info(ratings);")
ratings_info = cur.fetchall()
print_table_info("ratings", ratings_info)

# Get and print information for the 'links' table
cur.execute("PRAGMA table_info(links);")
links_info = cur.fetchall()
print_table_info("links", links_info)

# Get and print information for the 'tags' table
cur.execute("PRAGMA table_info(tags);")
tags_info = cur.fetchall()
print_table_info("tags", tags_info)


Table: movies
0: movieId: INT
1: title: TEXT
2: year: INT
3: genres: TEXT

Table: ratings
0: userId: INT
1: movieId: INT
2: rating: REAL
3: timestamp: INT

Table: links
0: movieId: INT
1: imdbId: TEXT
2: tmdbId: TEXT

Table: tags
0: userId: INT
1: movieId: INT
2: tag: TEXT
3: timestamp: NUM


In [ ]:
#For SQLite JDBC driver, it can be downloaded via:

!curl -O https://repo1.maven.org/maven2/org/xerial/sqlite-jdbc/3.34.0/sqlite-jdbc-3.34.0.jar

In [5]:
# install sqlite-jdbc-3.34.0.jar to the spark/jars directory

!sudo cp sqlite-jdbc-3.34.0.jar spark-3.1.1-bin-hadoop3.2/jars/

#check the jar file is in the spark/jars directory

!ls spark-3.1.1-bin-hadoop3.2/jars/

HikariCP-2.5.1.jar
JLargeArrays-1.5.jar
JTransforms-3.1.jar
RoaringBitmap-0.9.0.jar
ST4-4.0.4.jar
accessors-smart-1.2.jar
activation-1.1.1.jar
aircompressor-0.10.jar
algebra_2.12-2.0.0-M2.jar
antlr-runtime-3.5.2.jar
antlr4-runtime-4.8-1.jar
aopalliance-1.0.jar
aopalliance-repackaged-2.6.1.jar
arpack_combined_all-0.1.jar
arrow-format-2.0.0.jar
arrow-memory-core-2.0.0.jar
arrow-memory-netty-2.0.0.jar
arrow-vector-2.0.0.jar
audience-annotations-0.5.0.jar
automaton-1.11-8.jar
avro-1.8.2.jar
avro-ipc-1.8.2.jar
avro-mapred-1.8.2-hadoop2.jar
bonecp-0.8.0.RELEASE.jar
breeze-macros_2.12-1.0.jar
breeze_2.12-1.0.jar
cats-kernel_2.12-2.0.0-M4.jar
chill-java-0.9.5.jar
chill_2.12-0.9.5.jar
commons-beanutils-1.9.4.jar
commons-cli-1.2.jar
commons-codec-1.10.jar
commons-collections-3.2.2.jar
commons-compiler-3.0.16.jar
commons-compress-1.20.jar
commons-configuration2-2.1.1.jar
commons-crypto-1.1.0.jar
commons-daemon-1.0.13.jar
commons-dbcp-1.4.jar
commons-httpclient-3.1.jar
commons-io-2.5.jar
commons-l

In [6]:
# create a dataframe with including userid, movieid, genre and rating via pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# Read the ratings table into a dataframe
ratings_df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:sqlite:Datasets/movielens-small.db") \
    .option("dbtable", "ratings") \
    .load()

# Read the movies table into a dataframe
movies_df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:sqlite:Datasets/movielens-small.db") \
    .option("dbtable", "movies") \
    .load()

# Read the links table into a dataframe
links_df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:sqlite:Datasets/movielens-small.db") \
    .option("dbtable", "links") \
    .load()

# Read the tags table into a dataframe
tags_df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:sqlite:Datasets/movielens-small.db") \
    .option("dbtable", "tags") \
    .load()

In [21]:
# Write a query to create a dataframe with including userid, movieid, genre and rating


# join ratings and movies dataframes on movieid column and create a new dataframe called ratings_movies_df via pyspark dataframe
ratings_movies_df = ratings_df.join(movies_df, on="movieId", how="left")

# show the userid, movieid, genre and rating columns of ratings_movies_df dataframe
ratings_movies_df.select("userId", "movieId", "genres", "rating").show(10)

#count the number of rows in ratings_movies_df dataframe
ratings_movies_df.count()


+------+-------+------+------+
|userId|movieId|genres|rating|
+------+-------+------+------+
|    14|     26| Drama|   4.0|
|    31|     26| Drama|   2.0|
|    51|     26| Drama|   4.0|
|    79|     26| Drama|   4.0|
|   156|     26| Drama|   4.0|
|   161|     26| Drama|   3.0|
|   203|     26| Drama|   4.0|
|   219|     26| Drama|   3.0|
|   220|     26| Drama|   2.5|
|   228|     26| Drama|   4.0|
+------+-------+------+------+
only showing top 10 rows



100023

In [56]:
# Count ratings for each movie, and list top 5 movies with the highest value

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Calculate the count of ratings for each movie
movie_rating_count_df = ratings_movies_df.groupBy("movieId").agg(
    F.count("rating").alias("rating_count")
)

# Join the top_movies_df with the movies_df dataframe to get the movie titles
top_movies_df = movie_rating_count_df.join(movies_df, on="movieId", how="left")

top_movies_df.select("movieId", "title", "rating_count").sort("rating_count", ascending=False).show(5, truncate=False)



+-------+-------------------------+------------+
|movieId|title                    |rating_count|
+-------+-------------------------+------------+
|593    |Silence of the Lambs, The|337         |
|318    |Shawshank Redemption, The|328         |
|296    |Pulp Fiction             |327         |
|480    |Jurassic Park            |324         |
|356    |Forrest Gump             |318         |
+-------+-------------------------+------------+
only showing top 5 rows



In [53]:
# Find and list top 5 most rated genres

genre_rating_count_df = ratings_movies_df.groupBy("genres").agg(
    F.count("rating").alias("rating_count")
)

genre_rating_count_df.sort("rating_count", ascending=False).show(5, truncate=False)


+--------------+------------+
|genres        |rating_count|
+--------------+------------+
|Drama         |7008        |
|Comedy        |6396        |
|Comedy|Romance|3877        |
|Drama|Romance |3121        |
|Comedy|Drama  |3000        |
+--------------+------------+
only showing top 5 rows



In [58]:
#Find and list top 5 most rated tags except null values

tags_rating_count_df = ratings_movies_df.join(tags_df, on="movieId", how="left").groupBy("tag").agg(
    F.count("rating").alias("rating_count")).filter(tags_df.tag.isNotNull())

tags_rating_count_df.sort("rating_count", ascending=False).show(5, truncate=False)





+------------+------------+
|tag         |rating_count|
+------------+------------+
|drama       |3542        |
|sci-fi      |3035        |
|twist ending|2998        |
|psychology  |2672        |
|crime       |2570        |
+------------+------------+
only showing top 5 rows



In [61]:
# By using timestamp from ratings table, provide top 5 most frequent users within a week

genre_rating_count_df = ratings_movies_df.groupBy("userId").agg(
    F.count("timestamp").alias("timestamp_count")
)

genre_rating_count_df.sort("timestamp_count", ascending=False).show(5, truncate=False)



+------+---------------+
|userId|timestamp_count|
+------+---------------+
|516   |2268           |
|384   |1412           |
|187   |1338           |
|31    |1283           |
|377   |1241           |
+------+---------------+
only showing top 5 rows



In [62]:
# Calculate average ratings for each genre, and plot average ratings of top 10 genres with descending order

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Calculate the average rating for each genre
genre_rating_avg_df = ratings_movies_df.groupBy("genres").agg(
    F.avg("rating").alias("rating_avg")
)

# Sort the dataframe by rating_avg column
genre_rating_avg_df.sort("rating_avg", ascending=False).show(10, truncate=False)


+--------------------------------------------------------+----------+
|genres                                                  |rating_avg|
+--------------------------------------------------------+----------+
|Animation|Documentary                                   |5.0       |
|Crime|Documentary|War                                   |5.0       |
|Adventure|Fantasy|Mystery                               |5.0       |
|Action|Adventure|Animation|Comedy|Fantasy|Mystery|Sci-Fi|5.0       |
|Crime|Horror|Mystery                                    |4.75      |
|Animation|Comedy|Horror|IMAX                            |4.5       |
|Comedy|Crime|Western                                    |4.5       |
|Adventure|Crime|Drama|Horror|Mystery                    |4.5       |
|Adventure|Comedy|Crime|Drama|Romance                    |4.5       |
|Action|Animation|Drama|Fantasy|Sci-Fi                   |4.5       |
+--------------------------------------------------------+----------+
only showing top 10 

## TASK 2 - Recommender Design

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [ ]:
spark = SparkSession.builder.appName("movieRecommendation").getOrCreate() 

In [ ]:
# using movie rating data to probide implicit feature using ALS(Alternate Least Squares)

movie_rating=[]

for row in cur.execute("""SELECT userId, movieId, rating 
                          FROM ratings
                                      """):
  movie_rating.append(row)

schema = ["userId", "movieId", "rating"]
 
# calling function to create dataframe
movie_rating_df = spark.createDataFrame(movie_rating, schema)

movie_rating_df




In [ ]:
# description of created schema 
movie_rating_df.printSchema()

In [ ]:
#splitting dataset to train the model as 80% for train and remaining for test data.
(train, test) = movie_rating_df.randomSplit([0.8, 0.2], seed=87)

In [ ]:
# 1st Recommender model - Alternating Least Square (ALS) Matrix Factorization in Collaborative Filtering on rating (as actual values) 

als = ALS(rank=10, maxIter=15, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")

model = als.fit(train)

pred = model.transform(test)

pred = pred.selectExpr("userId as userId","movieId as movieId","rating as rating","prediction as implicit")

pred.show(truncate=False)

In [ ]:
#calculating RMSE and MAE to evaluate performance of the models. 

eval_rmse = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="implicit")
eval_mae = RegressionEvaluator(metricName="mae", labelCol="rating", predictionCol="implicit")


rmse = eval_rmse.evaluate(pred)
mae = eval_mae.evaluate(pred)


print("RMSE of ALS:", rmse)
print("MAE of ALS:", mae)




In [ ]:
# 2nd Recommender model - Alternating Least Square (ALS) Matrix Factorization in Collaborative Filtering on designed implicit feedback values 


(train_implicit, test_implicit) = pred.randomSplit([0.8, 0.2], seed=87)

als_implicit = ALS(rank=10, maxIter=15, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="implicit", coldStartStrategy="drop")

model_implicit = als_implicit.fit(train_implicit)

pred_implicit = model_implicit.transform(test_implicit)

pred_implicit.show(truncate=False)

In [ ]:
eval_rmse = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
eval_mae = RegressionEvaluator(metricName="mae", labelCol="rating", predictionCol="prediction")


rmse = eval_rmse.evaluate(pred_implicit)
mae = eval_mae.evaluate(pred_implicit)


print("RMSE of ALS_Implicit Feedback:", rmse)
print("MAE of ALS_Implicit Feedback:", mae)

When compared these two models, the 1st model ( ALS on rating ) shows better performance compared to 2nd model (ALS on implicit feedback) according to error metrics such as Root Mean Square Error (RMSE) and Mean Absolute Error (MAE). 

## Task – 3 Text Analysis

In [ ]:
spark = SparkSession.builder.appName("textAnalysis").getOrCreate()

In [ ]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf


In [ ]:
# download the data in Dataset folder.



!curl -o Datasets/aclImdb_v1.tar.gz https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz


In [ ]:
#Extract the tar file as dataset

import tarfile

def tarfile_extract(tar_file, output_dir=os.getcwd()+'/Datasets'):
    tar = tarfile.open(tar_file, 'r:gz')
    total_files = sum(1 for _ in tar)
    tar.extractall(output_dir, members=extract_progress(tar, total_files))
    tar.close()

def extract_progress(tar, total_files):
    for member in tar:
        yield member
        total_files -= 1
        print(f"Remaining files: {total_files}", end='\r')
    print("\nExtraction completed.")

wd = os.getcwd()
tarfile_extract(wd+'/Datasets/aclImdb_v1.tar.gz')


In [ ]:

#using 'alldata' list to store all the files in the directories
alldata=[]


#collecting data in train/pos folder
for fname in os.listdir(wd+'/Datasets/aclImdb/train/pos'):
    with open(os.path.join(wd+'/Datasets/aclImdb/train/pos', fname), encoding = 'utf-8') as infile:
        for line in infile:
            alldata.append((line,'train','pos'))

#collecting data in train/neg folder
for fname in os.listdir(wd+'/Datasets/aclImdb/train/neg'):
    with open(os.path.join(wd+'/Datasets/aclImdb/train/neg', fname), encoding = 'utf-8') as infile:
        for line in infile:
            alldata.append((line,'train','neg'))
#collecting data in test/pos folder
for fname in os.listdir(wd+'/Datasets/aclImdb/test/pos'):
    with open(os.path.join(wd+'/Datasets/aclImdb/test/pos', fname), encoding = 'utf-8') as infile:
        for line in infile:
            alldata.append((line,'test','pos'))
#collecting data in test/neg folder
for fname in os.listdir(wd+'/Datasets/aclImdb/test/neg'):
    with open(os.path.join(wd+'/Datasets/aclImdb/test/neg', fname), encoding = 'utf-8') as infile:
        for line in infile:
            alldata.append((line,'test','neg'))




In [ ]:

from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType

appName = "list to Spark Data Frame"
master = "local"

# Create Spark session
spark = SparkSession.builder \
    .appName(appName) \
    .master(master) \
    .getOrCreate()

# List
data = alldata

# Create a schema for the dataframe
schema = StructType([
    StructField('content', StringType(), True),
    StructField('label', StringType(), True),
    StructField('sentiemtn', StringType(), True)
])

# Convert list to RDD
rdd = spark.sparkContext.parallelize(data)

# Create data frame
df = spark.createDataFrame(rdd,schema)
print(df.schema)
df.show()

In [ ]:
df.count()

In [ ]:
# Schema of created Spark Dataframe

df.printSchema()

###Tokenization

In [ ]:
#using RegexTokenizer for tokenizing contents

tokenizer = RegexTokenizer(inputCol="content", outputCol="tokenized_content", pattern="\\W") # used regexp to determine pattern as 'not word'

countTokens = udf(lambda w: len(w), IntegerType())

tokenized = tokenizer.transform(df)

tokenized.show()

### Removing Stop Words

In [ ]:
type(tokenized)

In [ ]:
from pyspark.ml.feature import StopWordsRemover

tokenized.show()

In [ ]:
df_tokenized = tokenized.select("tokenized_content").withColumn("tokenCount", countTokens(col("tokenized_content")))

In [ ]:
type(df_tokenized)

In [ ]:
# SWR -> stop word remover
SWR  = StopWordsRemover (inputCol='tokenized_content', outputCol='SWRed')


#See the result of removal operation
SWR.transform(df_tokenized).select('SWRed').show(truncate=False)